In [ ]:
import pandas as pd
import numpy as np

Metadata file
The metadata file has information about each Patch-seq cell, including its different identifiers and assigned cell types.

In [ ]:
metadata = pd.read_csv("downloads/20200711_patchseq_metadata_mouse.csv")

metadata.head()

Transcriptomic data
Here, we have already downloaded the (large) transcriptomic expression data file. In that file, expression is quantified as counts-per-million (cpm), and each row is a gene and each column is a cell.

import the transciptomic expression data
CHECK THIS FILE that it is the large file   

In [ ]:
gene_data = pd.read_csv(
    "downloads/20200513_Mouse_PatchSeq_Release_cpm.v2.csv",
    index_col=0,
)
gene_data

the select_markers.csv was not available and so need to be generated. I am trying this method

the select marker file is not provided
https://community.brain-map.org/t/question-on-patchseq-jupyter-example/1030

##Tried a few different ways to generate a select markers file. They are all similar and none show similar clustering to the jupyter notebook online.
This is not the focus of the course so move on to downloading DANDI file and exploring.

In [ ]:
import scanpy as sc

# Assuming gene_data is your DataFrame with genes as rows and cells as columns
adata = sc.AnnData(gene_data.T)

# Ensure indices match
metadata.index = adata.obs_names

## Assuming 'cell_type' is a column in adata.obs that contains cell type annotations
adata.obs['cell_type'] = metadata['T-type Label'].astype('category')  

## Filter out groups with fewer than a minimum number of samples (e.g., 3)
min_samples = 3
cell_type_counts = adata.obs['cell_type'].value_counts()
valid_cell_types = cell_type_counts[cell_type_counts >= min_samples].index
adata = adata[adata.obs['cell_type'].isin(valid_cell_types)]

# Log-transform the data
sc.pp.log1p(adata)

# # Find marker genes
sc.tl.rank_genes_groups(adata, 'cell_type', method='t-test')

# # Extract marker genes
marker_genes = pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(50)

# # Save to CSV
marker_genes.to_csv('downloads/select_markers.csv', index=False)

In [ ]:
print(adata.obs['cell_type'].value_counts())
# print(adata)
# print(metadata['T-type Label'].value_counts())
print(metadata['T-type Label'].index[:10])


print(adata.obs_names[:10])  # Check the first 10 cell IDs in adata
print(metadata.index[:10]) 

use select markers to generate umap

In [ ]:
#method1
import pandas as pd
import umap
import numpy as np

# Read the CSV file
marker_genes_for_umap = pd.read_csv("downloads/select_markers.csv")

# Melt the DataFrame to have a single column of gene names
marker_genes_for_umap = marker_genes_for_umap.melt(value_name="Gene").dropna()["Gene"]

# Ensure the gene names are unique
marker_genes_for_umap = marker_genes_for_umap.drop_duplicates()

# Check if the gene names exist in gene_data
missing_genes = marker_genes_for_umap[~marker_genes_for_umap.isin(gene_data.index)]
if not missing_genes.empty:
    print(f"Warning: The following genes are missing in gene_data: {missing_genes.tolist()}")

# Proceed with UMAP embedding
embedding = umap.UMAP(n_neighbors=25).fit_transform(
    np.log2(gene_data.loc[marker_genes_for_umap, :].values.T + 1)
)



In [ ]:
#method 2
import pandas as pd
import umap
import numpy as np

# Read the CSV file
marker_genes_for_umap = pd.read_csv("downloads/select_markers.csv")

# Initialize an empty list to store the genes
ordered_genes = []

# Iterate through each cell type column
for cell_type in marker_genes_for_umap.columns:
    genes = marker_genes_for_umap[cell_type].dropna().tolist()
    ordered_genes.extend(genes)

# Ensure the gene names are unique while maintaining order
ordered_genes = pd.unique(ordered_genes)

# Check if the gene names exist in gene_data
missing_genes = [gene for gene in ordered_genes if gene not in gene_data.index]
if missing_genes:
    print(f"Warning: The following genes are missing in gene_data: {missing_genes}")

# Proceed with UMAP embedding
embedding = umap.UMAP(n_neighbors=25).fit_transform(
    np.log2(gene_data.loc[ordered_genes, :].values.T + 1)
)


both methods produce same umap

Plot the umap

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline  
plt.figure(figsize=(8, 8))
plt.scatter(*embedding.T, s=1, edgecolor="none")
sns.despine()